In [2]:
import numpy as np
from pyxtal import Group, Wyckoff_position
from cctbx.sgtbx.direct_space_asu.reference_table import get_asu

group = 196
letter = 'h'
asu = get_asu(group)

pt = (0.1, 0.05, 0.07)
wp = Wyckoff_position.from_group_and_letter(group, letter)

offset = -np.floor(wp.apply_ops(pt))

In [3]:
successes = 0
bmin = np.array(asu.box_min())
bmax = np.array(asu.box_max())
for _ in range(1000):    
    pt2 = bmin + np.random.random(3) * (bmax - bmin)
    if asu.is_inside(pt2):        
        if not np.allclose(offset, -np.floor(wp.apply_ops(pt))):
            raise ValueError('Well shit')
        else:
            successes += 1

To do: 

- Intersect the ASU with the Wyckoff point general position
- How to do unconstrained generation on a generic solid?
- Check that the modulo stuff doesn't matter for anything within the ASU

In [4]:
successes

175

In [5]:
import plotly.express as px
import plotly.graph_objects as go
from scipy.spatial import Delaunay, ConvexHull

group = 225
asu = get_asu(group)

verts = np.array(asu.shape_vertices()).astype(float)

import matplotlib.pyplot as plt
xyverts = verts[:, [0, 1]]
xyhull = ConvexHull(xyverts)

# plt.scatter(*xyverts.T)
# for simplex in xyhull.simplices:
#     plt.plot(*xyverts[simplex].T)

import networkx as nx

edges = []
for i, j in xyhull.simplices:
    xi, xj = xyverts[:, 0][[i, j]]    
    if xi == xj:
        # can only happen at end or start, ignore
        pass
    else:
        if xi > xj:
            i, j = j, i
        edges.append((i, j))

xygraph = nx.DiGraph(edges)

# nx.draw_networkx(xygraph, {i: p for i, p in enumerate(xyhull.points)})

def find_path(op):
    vertx = xyhull.points[:, 0]
    verty = xyhull.points[:, 1]

    inits = [v for v in xyhull.vertices if vertx[v] == min(vertx)]

    start = op(inits, key=lambda v: verty[v])
    path = [start]
    neighbors = list(xygraph.neighbors(start))
    while neighbors:
        next_pt = op(neighbors, key=lambda v: verty[v])
        path.append(next_pt)
        neighbors = list(xygraph.neighbors(next_pt))

    return path

min_path = find_path(min)
max_path = find_path(max)
# plt.plot(*xyverts[min_path].T)
# plt.plot(*xyverts[max_path].T)

import scipy.interpolate as interp

ymin = interp.interp1d(*xyverts[min_path].T)
ymax = interp.interp1d(*xyverts[max_path].T)
def to_bounded_y(x, y):
    lo = ymin(x)
    hi = ymax(x)
    return lo + y * (hi - lo)

xx, yy = np.mgrid[0:0.5:20j, 0:1:20j]
xx = xx.flatten()
yy = yy.flatten()
# plt.plot(*xyverts[min_path].T)
# plt.plot(*xyverts[max_path].T)
# plt.scatter(xx, to_bounded_y(xx, yy), c=xx + yy)

import scipy.interpolate as interp

ymin = interp.interp1d(*xyverts[min_path].T)
ymax = interp.interp1d(*xyverts[max_path].T)
def to_bounded_y(x, y):
    lo = ymin(x)
    hi = ymax(x)
    return lo + y * (hi - lo)

xx, yy = np.mgrid[0:0.5:20j, 0:1:20j]
xx = xx.flatten()
yy = yy.flatten()
# plt.plot(*xyverts[min_path].T)
# plt.plot(*xyverts[max_path].T)
# plt.scatter(xx, to_bounded_y(xx, yy), c=xx + yy)




N = np.array([cut.as_float_cut_plane().n for cut in asu.cuts])
c = np.array([cut.as_float_cut_plane().c for cut in asu.cuts])
def get_z_bounds(x, y):
    # Nr + c >= 0

    # ((N @ np.array([[0, 0, 0.1]]).T).flatten() + c) >= 0    
    c_xy = N[:, :2] @ np.vstack([x, y]) + c.reshape(-1, 1)
    N_xy = N[:, [2]]

    N_xy, c_xy = N_xy[abs(N_xy.reshape(-1)) > 1e-6], c_xy[abs(N_xy.reshape(-1)) > 1e-6]
    # N_xy * z + c_xy >= 0
    # if N_xy > 0, z >= -c_xy / N_xy
    # else, z <= -c_xy / N_xy

    z_vals = -c_xy / N_xy
    zmin = z_vals[N_xy.reshape(-1) > 0].max(axis=0)
    zmax = z_vals[N_xy.reshape(-1) < 0].min(axis=0)
    return (zmin, zmax)

zmins, zmaxs = get_z_bounds(*xyverts.T)
zmin = interp.LinearNDInterpolator(xyverts, zmins)
zmax = interp.LinearNDInterpolator(xyverts, zmaxs)

def get_unbounded_xyz(x_u, y_u, z_u):
    xmin, xmax = xyverts[:, 0].min(), xyverts[:, 0].max()
    x = xmin + x_u * (xmax - xmin)
    y = to_bounded_y(x, y_u)
    zlo = zmin(np.vstack([x, y]).T)
    zhi = zmax(np.vstack([x, y]).T)
    z = zlo + z_u * (zhi - zlo)
    return (x, y, z)

xx_u, yy_u, zz_u = np.mgrid[0:1:20j, 0:1:10j, 0:1:10j]
xx_u = xx_u.flatten()
yy_u = yy_u.flatten()
zz_u = zz_u.flatten()

x, y, z = get_unbounded_xyz(xx_u, yy_u, zz_u)


fig = px.scatter_3d(None, *verts.T)
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=3)))
fig.update_layout(scene={f'{a}axis': dict(range=(-0.5, 0.5)) for a in 'xyz'})

fig

In [87]:
wp = Group(group).Wyckoff_positions[0]
def inv(x1, y1, z1):
    x = y = z = None    
    for pos in wp.apply_ops([x1, y1, z1]) % 1:
        if asu.is_inside(pos):
            x, y, z = pos
            break
    if x is None:
        raise ValueError('Oops')
    xmin, xmax = xyverts[:, 0].min(), xyverts[:, 0].max()
    x_u = (x - xmin) / (xmax - xmin)
    ylo = ymin(x)
    yhi = ymax(x)    
    y_u = (y - ylo) / (yhi - ylo)

    zlo = zmin(np.vstack([x, y]).T)[0]
    zhi = zmax(np.vstack([x, y]).T)[0]
    z_u = (z - zlo) / (zhi - zlo)
    return np.array([np.array([x_u]), np.array([y_u]), np.array([z_u])])



x1, y1, z1 = np.random.uniform(size=3)
wp.are_equivalent_pts(np.array(get_unbounded_xyz(*inv(x1, y1, z1))).flatten(), (x1, y1, z1))

True

In [82]:
asu.box_min()

[0, 0, 0]

In [89]:
get_unbounded_xyz(0.26, 1, 1)

(0.13, 0.13, array([0.13]))

In [ ]:
inv(0.13, 0.13, 0.13))

In [90]:
wpk = Wyckoff_position.from_group_and_letter(225, 'f')
wpk.get_all_positions([0.13, 0.13, 0.13])

array([[0.79666667, 0.79666667, 0.79666667],
       [0.20333333, 0.20333333, 0.79666667],
       [0.20333333, 0.79666667, 0.20333333],
       [0.79666667, 0.20333333, 0.20333333],
       [0.79666667, 0.79666667, 0.20333333],
       [0.20333333, 0.20333333, 0.20333333],
       [0.79666667, 0.20333333, 0.79666667],
       [0.20333333, 0.79666667, 0.79666667],
       [0.79666667, 0.29666667, 0.29666667],
       [0.20333333, 0.70333333, 0.29666667],
       [0.20333333, 0.29666667, 0.70333333],
       [0.79666667, 0.70333333, 0.70333333],
       [0.79666667, 0.29666667, 0.70333333],
       [0.20333333, 0.70333333, 0.70333333],
       [0.79666667, 0.70333333, 0.29666667],
       [0.20333333, 0.29666667, 0.29666667],
       [0.29666667, 0.79666667, 0.29666667],
       [0.70333333, 0.20333333, 0.29666667],
       [0.70333333, 0.79666667, 0.70333333],
       [0.29666667, 0.20333333, 0.70333333],
       [0.29666667, 0.79666667, 0.70333333],
       [0.70333333, 0.20333333, 0.70333333],
       [0.

In [97]:
wpk.get_frozen_axis()

[1, 2]

In [99]:
import pickle

with open('all_wps.pkl', 'rb') as all_wps_pkl:
    all_wps = pickle.load(all_wps_pkl)

# x never depends on y and z
for wp in all_wps:
    gp = wp.gen_pos()
    rot = gp.rotation_matrix
    if np.allclose(rot[:, 0], 0):
        if not np.allclose(rot[0], 0):
            raise ValueError('Whoops')

In [104]:
# y never depends on z
for wp in all_wps:
    gp = wp.gen_pos()
    rot = gp.rotation_matrix
    if np.allclose(rot[:, 1], 0):
        if not np.allclose(rot[[1, 2], [2, 1]], 0):
            raise ValueError('Whoops')